# Optimization

One of the main applications of **CADET-Process** is performing optimization studies.
Optimization refers to the selection of a solution with regard to some criterion.
In the simplest case, an optimization problem consists of minimizing some function $f(x)$ by systematically varying the input values $x$ and computing the value of that function.

$$
\min_x f(x)
$$

In the context of physico-chemical processes, examples for the application of optimization studies include scenarios such as process optimization and parameter estimation.
Here, often many variables are subject to optimization, multiple criteria have to be balanced, and additional linear and nonlinear constraints need to be considered.

$$
\min_x f(x) \\
s.t. \\
    g(x) \le 0, \\
    h(x) = 0, \\
    x \in \mathbb{R}^n \\
$$


where $g$ summarizes all inequality constraint functions, and $h$ equality constraints.

In the following, the `optimization` module of **CADET-Process** is introduced. To decouple the problem formulation from the problem solution, two classes are provided:
- An `OptimizationProblem` class to specify optimization variables, objectives and constraints, and
- an abstract `Optimizer` interface which allows using different external optimizers to solve the problem.

## Example 1: Minimization of a quadratic function

Usually, the objective function is not known; it can only be evaluated at certain points.
For demonstration purpouses, consider a quadratic function to be minimized.

$$
f(x) = x^2
$$

Since we already know a lot about this function, it can help to introduce some of the Optimization concepts of CADET-Process.
For example, the results should yield:
- $x_{opt} = 0$
- $f_{opt} = 0$.

### OptimizationProblem

The `OptimizationProblem` class is is used to specify optimization variables, objectives and constraints.
After import, the `OptimizationProblem` initialized with a name.

#### Optimization Variables
Any number of variables can be added to the `OptimizationProblem`.
To add a variable, use the `add_variable` method.
In this case, there is only a single variable.
The first argument is the name of the variable.
Moreover, lower and upper bounds can be specified.

The total number of variables is stored in `n_variables` and the names in `variable_names`

### Objectives
Any `callable` (i.e. an object that can be called using the `( )` operator) can be added as an objective as long as it takes x as the first argument.
Multi-objective optimization is also possible with CADET-Python (more on that later).
For now, the objective must return a single, scalar value.

```{note}
Usually, there are multiple variables involved. Hence, the function is expected to accept a list.
```

To evaluate the this function, the `evaluate_objective` method can be used.
This is useful to test whether everything works as expected.

If the value is out of bounds, an error will be thrown.

### Optimizer
The `OptimizerAdapter` provides a unified interface for using external optimization libraries.
It is responsible for converting the `OptimizationProblem` to the specific API of the external `Optimizer`.
Currently, adapters to **Pymoo** and **Scipy's** optimization suite are implemented, all of which are published under open source licenses that allow for academic as well as commercial use.

Before the optimization can be run, the `Optimizer` needs to be initialized and configured.
For this example, `U_NSGA3` is used, a genetic algorithm.

All options can be displayed the following way:

To reduce the calculation time, let's limit the maximum number of generations that the `Optimizer` evaluates:

To optimize the `OptimizationProblem`, call the `optimize()` method.
By default, CADET-Process tries to autogenerate initial values.
However, it's also possible to pass them as an additional `x0` argument.
More on generating initial values later.

### Optimization Progress and Results

The `OptimizationResults` which are returned contain information about the progress of the optimization.
For example, the attributes `x` and `f` contain the final value(s) of parameters and the objective function.

After optimization, several figures can be plotted to vizualize the results.
For example, the convergence plot shows how the function value changes with the number of evaluations.

The `plot_objectives` method shows the objective function values of all evaluated individuals.
Here, lighter color represent later evaluations.
Note that by default the values are plotted on a log scale if they span many orders of magnitude.
To disable this, set `autoscale=False`.

Note that more figures are created for constrained optimization, as well as multi-objective optimization.
All figures are also saved automatically in the `working_directory`.
Moreover, results are stored in a `.csv` file.
- The `results_all.csv` file contains information about all evaluated individuals.
- The `results_last.csv` file contains information about the last generation of evaluated individuals.
- The `results_pareto.csv` file contains only the best individual(s).

## Example 2: Constrained Optimization

Example taken from [SciPy Documentation](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#id34)

As an example let us consider the constrained minimization of the Rosenbrock function:

$$
\min_{x_0, x_1} & ~~100\left(x_{1}-x_{0}^{2}\right)^{2}+\left(1-x_{0}\right)^{2} &\\
    \text{subject to: } & x_0 + 2 x_1 \leq 1 & \\
                        & x_0^2 + x_1 \leq 1  & \\
                        & x_0^2 - x_1 \leq 1  & \\
                        & 2 x_0 + x_1 = 1 & \\
                        & 0 \leq  x_0  \leq 1 & \\
                        & -0.5 \leq  x_1  \leq 2.0. &
$$

This optimization problem has the unique solution $[x_0, x_1] = [0.4149,~ 0.1701]$.

```{figure} ./figures/rosenbrock.png
:align: center
:width: 50%

```

To setup this problem, first a new `OptimizationProblem` is created and the variables are added, including bounds.
It is important to note, that `x0` cannot be used as variable name since it is reserved for the initial values.

Then, then objective function is defined and added.

In [ ]:
def rosenbrock_objective(x):
    x_0 = x[0]
    x_1 = x[1]

    return 100 * (x_1 - x_0 ** 2) ** 2 + (1 - x_0) ** 2


rosenbrock_problem.add_objective(rosenbrock_objective)

### Linear inequality constraints
Linear constraints are usually defined in the following way

$$
A \cdot x \leq b
$$

In **CADET-Process**, add each row $a$ of the constraint matrix needs to be added individually.
The `add_linear_constraint` function takes the variables subject to the constraint as first argument.
The left-hand side $a$ and the bound $b_a$ are passed as second and third argument.
It is important to note that the column order in $a$ is inferred from the order in which the optimization variables are passed.

To add the constraints of the Rosenbrock function to the optimization problem, add the following:

To wheck if a point fulfils the linear inequality constraints, use the `check_linear_constraints` method.
It returns `True` if the point is within bounds and `False` otherwise.

### Linear equality constraints
Linear equality constraints are usually defined in the following way

$$
A_{eq} \cdot x = b_{eq}
$$

In **CADET-Process**, add each row $a_{eq}$ of the constraint matrix needs to be added individually.
The `add_linear_equality_constraint` function takes the variables subject to the constraint as first argument.
The left-hand side $a_{eq}$ and the bound $b_{eq, a}$ are passed as second and third argument.
It is important to note that the column order in $a$ is inferred from the order in which the optimization variables are passed.

To add this constraint of the Rosenbrock function

$$
2 x_0 + x_1 = 1
$$

to the optimization problem, add the following:

To wheck if a point fulfils the linear equality constraints, use the `check_linear_equality_constraints` method.
It returns `True` if the point is within bounds and `False` otherwise.

### Nonlinear constraints
It is also possible to add nonlinear constraints to the `OptimizationProblem`.

$$
g(x) \le 0 \\
$$

In contrast to linear constraints, and analogous to objective functions, nonlinear constraints need to be added as a callable functions.
Note that multiple nonlinear constraints can be added.
In addition to the function, lower or upper bounds can be added.

To add the constraints of the Rosenbrock function to the optimization problem, add the following.

Again, the function can be evaluated manually.

### Optimizer
To solve this problem, a **trust region method** is used, here:

### Optimization Progress and Results

Since in this problem, nonlinear constraints are involved, their convergence can also be plotted

### Initial Values

To start solving any optimization problem, initial values are required.
In CADET-Process, this can be done automatically if
- all variables have upper and lower bounds
- no linear equality constraints exist (this will be improved in the future!)

To create initial values, call `create_initial_values` and specify the number of individuals that should be returned.
By default, a random value is returned which fulfills all bound constraints and linear constraints.

In [ ]:
if rosenbrock_problem.n_linear_equality_constraints > 0:
    rosenbrock_problem.remove_linear_equality_constraint(0) # just for demonstration purposes

In [ ]:
x0 = rosenbrock_problem.create_initial_values(method='chebyshev')
print(x0)

By specifying `method='chebyshev'`, the so-called Chebyshev center is returned; the center of a minimal-radius ball enclosed by the convex parameter space.
If no information about the location is known, this can be a good starting point.

In [ ]:
x0 = rosenbrock_problem.create_initial_values(method='chebyshev')
print(x0)

For population based algorithms such as genetic algorithm, an entire population is required.
Because efficiently sampling a convex polytope can be difficult, we make use  of [hopsy](https://modsim.github.io/hopsy/index.html), a tool for Markov chain Monte Carlo sampling on convex polytopes.

Let's first create a method to better visualize these points

In [ ]:
def plot_initial_values(x0):
    import matplotlib.pyplot as plt
    import numpy as np
    fig, ax = plt.subplots()
    try:
        ax.scatter(x0[:, 0], x0[:, 1])
        ax.set_xlabel(r'$x_0$')
        ax.set_ylabel(r'$x_1$')
    except IndexError:
        ax.scatter(x0, np.ones_like((x0)))
        ax.set_xlabel(r'$x_0$')
    fig.tight_layout()

x0 = rosenbrock_problem.create_initial_values(500)
plot_initial_values(x0)